In [ ]:
import os
import torch
import pickle
import argparse
import h5py
from torch.autograd import Variable
from model import NetD, NetG
from PIL import Image, ImageDraw
import torchvision.transforms as transforms

In [ ]:
def concat_image(images,nrows, ncols):
    width = 128
    height = 128
    
    background = Image.new('RGBA',(128*ncols, 128*nrows), (255, 255, 255, 255))
    bg_w, bg_h = background.size
    
    for i,image in enumerate(images):
    #print(i)
        img = image
        draw = ImageDraw.Draw(img, "RGB")
        #font = ImageFont.truetype("/Library/Fonts/Arial.ttf",50)
        #draw.text((50, 50), str(i),)
        start_X = i%ncols * 128
        start_Y = i//ncols * 128
        offset = (start_X,start_Y,start_X+width,start_Y+height)
        background.paste(img,offset)
        
    return background

In [ ]:
h = h5py.File('./data/datasets/products/train/data.h5py','r')['train']

In [ ]:
for i in h.keys():
    print(i)

In [ ]:
h['asin'][:10]

In [ ]:
h['cate'][:10]

In [ ]:
checkpoint_path = 'outputs/netg_checkpoints/netG__epoch_45.pth'

In [ ]:
n_z = 100
n_l = 100
n_t = 500
n_c = 64
netG = NetG(n_z=n_z, n_l=n_l, n_c=n_c, n_t=n_t)
netG.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),])

In [ ]:
netG.zero_grad()

### train sample

In [ ]:
index = 3
caption = Variable(torch.from_numpy(h['docvec'][index].reshape(1,-1)))
noise = Variable(torch.randn(1, 100)) # create random noise
noise.data.normal_(0,1) # normalize the noise
fake = netG(noise, caption)

In [ ]:
img = transform(fake[0].data)

In [ ]:
img

### val sample

### test sample

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/datasets/products/products.tsv',sep='\t',header=None)
df.info()

In [ ]:
df[1].value_counts()

In [ ]:
sub = df[df[1] == 'Movies&TV>Movies']
imgdir = 'data/datasets/products/images/'
images = []
for i, row in sub.iterrows():
    asin = row[0]
    images.append(Image.open( imgdir+ asin +'.jpg').resize((128,128)))
    if i == 10:
        break

In [ ]:
concat_image(images,1,len(images))